# Final Test

In [2]:
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Set OpenAI API key
import openai
import config
api_key = config.api_key
openai.api_key = api_key

# Also set as environment variable for LangChain
os.environ["OPENAI_API_KEY"] = api_key

# Step 1: Load and split web content from TEQ3.AI website
urls = [
    "https://www.teq3.ai/",
    "https://www.teq3.ai/about-us",
    "https://www.teq3.ai/courses",
    "https://www.teq3.ai/programs", 
    "https://www.teq3.ai/data-analytics",
    "https://www.teq3.ai/artificial-intelligence",
    "https://www.teq3.ai/contact",
    "https://www.teq3.ai/services"
]

# Load documents silently (no print statements)
all_documents = []

for url in urls:
    try:
        loader = WebBaseLoader(url)
        documents = loader.load()
        all_documents.extend(documents)
    except Exception as e:
        # Only print if there's an error
        print(f"Error loading {url}: {str(e)}")

documents = all_documents

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Step 2: Create embeddings and vector store
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
vectorstore = FAISS.from_documents(texts, embeddings)

# Step 3: Initialize the LLM with more conversational settings
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.8, max_tokens=300, openai_api_key=api_key)

# Step 4: Set up conversational memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Step 5: Create the conversational retrieval chain with ChatGPT-style persona
custom_prompt = """
You are TEQ3's AI Assistant - a friendly, helpful, and intuitive chatbot that acts like ChatGPT but specializes in TEQ3's AI and Data Analytics training programs.

PERSONALITY TRAITS:
- Be conversational, warm, and genuinely helpful
- Show enthusiasm about AI and data careers
- Use emojis occasionally to be friendly 😊
- Ask follow-up questions to better understand user needs
- Be encouraging and supportive about career transitions
- Provide detailed, thoughtful responses like ChatGPT would

CAPABILITIES:
- Answer questions about TEQ3's courses, programs, and services
- Provide career guidance and advice
- Help with general AI and data science questions
- Assist with enrollment and program selection
- Handle technical support routing
- Engage in friendly conversation while staying helpful

RESPONSE STYLE:
- Start with a warm greeting or acknowledgment
- Provide comprehensive, helpful answers
- End with a relevant follow-up question or offer of assistance
- Be conversational and natural, not robotic
- Show personality while remaining professional

If you don't know something specific about TEQ3, be honest and offer to connect them with the right person.

Context: {context}
Chat History: {chat_history}
User: {question}

TEQ3 AI Assistant:
"""

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PromptTemplate(
        input_variables=["context", "chat_history", "question"],
        template=custom_prompt
    )}
)

# Enhanced query analysis functions
def categorize_query(query):
    """Categorize user queries for smart routing"""
    query_lower = query.lower()
    
    # Technical support keywords
    technical_keywords = [
        "payment", "purchase", "buying", "can't buy", "payment failed", 
        "credit card", "billing", "refund", "login", "password", "access",
        "platform", "website", "technical", "error", "bug", "not working",
        "loading", "broken", "system", "account", "sign in", "log in", "can't access"
    ]
    
    # Complaint/concern keywords
    complaint_keywords = [
        "complaint", "complain", "issue", "problem", "concern", "worried",
        "dissatisfied", "disappointed", "frustrated", "unhappy", "feedback",
        "poor", "bad", "terrible", "awful", "wrong", "mistake", "unsatisfied"
    ]
    
    # Career guidance keywords
    career_keywords = [
        "which course", "what should i", "career change", "career transition",
        "job prospects", "salary", "right for me", "help me choose", "confused",
        "not sure", "advice", "recommend", "best option", "career path",
        "switch to tech", "enrollment decision", "course selection", "ai career",
        "data career", "job guarantee", "placement"
    ]
    
    # Consultant interest keywords
    consultant_keywords = [
        "yes", "sure", "interested", "consultant", "career advisor", 
        "speak to someone", "human", "person", "advisor", "guidance",
        "help me decide", "connect me", "arrange", "schedule", "talk to",
        "call me", "consultation"
    ]
    
    if any(keyword in query_lower for keyword in technical_keywords):
        return "technical"
    elif any(keyword in query_lower for keyword in complaint_keywords):
        return "complaint"
    elif any(keyword in query_lower for keyword in career_keywords):
        return "career_guidance"
    elif any(keyword in query_lower for keyword in consultant_keywords):
        return "consultant_interest"
    else:
        return "general"

def handle_technical_support():
    """Provide technical support contact information"""
    return """I understand you're experiencing a technical issue! 🛠️ Don't worry, our technical support team is here to help.

Our tech support can assist with:
   • Payment and billing problems 💳
   • Account access issues 🔐
   • Platform technical difficulties 💻
   • Purchase and enrollment problems 📝

Here's how to get help:
   📧 Email: support@teq3.ai
   📞 Phone: [Available on our website]
   💬 Live Chat: Check teq3.ai for instant support
   ⏰ Support Hours: We're here when you need us!

For the fastest response, I'd recommend checking our website for live chat options. Is there anything else I can help you with while you're here? 😊"""

def handle_complaint():
    """Handle complaints with appropriate routing"""
    return """I'm really sorry to hear about your concern! 😔 Your feedback is super important to us, and I want to make sure you get the best possible help.

Let me connect you with the right team:

🎯 **For course or program concerns:**
   📧 careers@teq3.ai | 📞 [Career team number]
   They're amazing at addressing program questions and providing personalized guidance!

🔧 **For technical, billing, or platform issues:**
   📧 support@teq3.ai | 📞 [Tech support number]
   Our tech wizards can sort out any platform problems

📋 **For general feedback or escalated concerns:**
   📧 hello@teq3.ai | 📞 [Main contact number]
   Direct line to our customer care team

Would you like me to help you get connected with the most appropriate team? I'm here to make sure you get the support you deserve! 💪"""

def handle_career_consultation():
    """Provide career consultation information"""
    return """That's fantastic! 🌟 I'm excited to help you connect with one of our AI career consultants - they're absolute experts at guiding people into amazing tech careers!

Here's how to reach our career consultation team:
   🌐 Visit: teq3.ai/contact
   📧 Email: careers@teq3.ai  
   📞 Phone: [Career consultation number]
   📋 Or fill out our consultation request form on our website

Our consultants are incredible at providing:
   🎯 Personalized course selection based on your goals
   📈 Career transition planning and strategy
   💰 Job market insights and salary expectations  
   🏆 Portfolio development and project guidance
   🤝 Industry networking and job search support
   ✅ Leveraging our 100% Job Guarantee program

They offer consultations via phone, video call, or even in-person if you're local! 

What specific area are you most interested in - AI, Data Analytics, or still exploring your options? 🤔"""

def suggest_career_consultation():
    """Smart suggestion for career consultation"""
    suggestions = [
        "Would you like to chat with one of our AI career consultants? They're amazing at helping people find their perfect tech path! 🚀",
        "Our career consultants are like career GPS systems - they can guide you to exactly where you want to go in tech! Want me to connect you? 🗺️",
        "Since this is such an exciting career decision, would you benefit from a personalized chat with our career experts? They love helping people like you! 💡",
        "I can arrange a consultation with our career team - they're fantastic at turning career dreams into reality! Interested? ✨"
    ]
    import random
    return random.choice(suggestions)

def run_chatbot():
    print("\n" + "🤖" + "="*58 + "🤖")
    print("         Welcome to TEQ3's AI Assistant! 🚀")
    print("🤖" + "="*58 + "🤖")
    print("Hey there! 👋 I'm your friendly TEQ3 AI Assistant!")
    print("I'm here to help you explore our amazing AI & Data programs:")
    print("   🤖 Artificial Intelligence | 📊 Data Analytics | 🎯 100% Job Guarantee")
    print("\n✨ What I can help you with:")
    print("   💬 Course & Career Questions → Let's chat!")
    print("   🛠️ Technical Issues → I'll connect you with our tech team")
    print("   📞 Want to talk to a human? → I'll set that up!")
    print("   🎓 Career Guidance → I'll get you expert advice")
    print("\nI'm designed to be helpful, friendly, and intuitive - just like ChatGPT! 😊")
    print("Ask me anything about AI careers, our courses, or just say hi!")
    print("\nType 'exit' to quit anytime")
    print("🤖" + "="*58 + "🤖" + "\n")
    
    # Initial greeting
    print("TEQ3 AI Assistant: Hi there! 👋 What brings you to TEQ3 today? I'm here to help with anything you need! 😊\n")
    
    while True:
        customer_query = input("You: ")
        
        if customer_query.lower() in ["exit", "quit", "bye", "goodbye"]:
            print("\nTEQ3 AI Assistant: It's been wonderful chatting with you! 🌟")
            print("   Don't forget to visit teq3.ai to start your AI journey!")
            print("   Remember: We offer 100% Job Guarantee! 🎯")
            print("   Take care and hope to see you soon! 👋✨\n")
            break
        
        # Categorize the query for smart routing
        query_category = categorize_query(customer_query)
        
        # Handle different query types
        if query_category == "consultant_interest":
            print("TEQ3 AI Assistant:", handle_career_consultation())
            print()
            continue
            
        elif query_category == "technical":
            print("TEQ3 AI Assistant:", handle_technical_support())
            print()
            continue
            
        elif query_category == "complaint":
            print("TEQ3 AI Assistant:", handle_complaint())
            print()
            continue
        
        # For general and career guidance queries, use the LLM chain
        try:
            response = chain({"question": customer_query})["answer"]
            print("TEQ3 AI Assistant:", response)
            
            # Smart follow-up suggestions based on query category
            if query_category == "career_guidance":
                print("\n💡", suggest_career_consultation())
                print("   Just say 'yes' or 'consultant' if you'd like personalized guidance!")
                
        except Exception as e:
            print("TEQ3 AI Assistant: Oops! 😅 I encountered a little hiccup processing that.")
            print("   No worries though! Let me connect you with our amazing support team:")
            print(handle_technical_support())
        
        print()

if __name__ == "__main__":
    run_chatbot()


🤖==========================================================🤖
         Welcome to TEQ3's AI Assistant! 🚀
🤖==========================================================🤖
Hey there! 👋 I'm your friendly TEQ3 AI Assistant!
I'm here to help you explore our amazing AI & Data programs:
   🤖 Artificial Intelligence | 📊 Data Analytics | 🎯 100% Job Guarantee

✨ What I can help you with:
   💬 Course & Career Questions → Let's chat!
   🛠️ Technical Issues → I'll connect you with our tech team
   📞 Want to talk to a human? → I'll set that up!
   🎓 Career Guidance → I'll get you expert advice

I'm designed to be helpful, friendly, and intuitive - just like ChatGPT! 😊
Ask me anything about AI careers, our courses, or just say hi!

Type 'exit' to quit anytime
🤖==========================================================🤖

TEQ3 AI Assistant: Hi there! 👋 What brings you to TEQ3 today? I'm here to help with anything you need! 😊

TEQ3 AI Assistant: Hello there! I am TEQ3's AI Assistant. How may I assist you to